In [271]:
#Import packages
from datetime import datetime, timedelta
from datetime import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [272]:
#Read the data file
data=pd.read_csv('assignment-data.csv')

In [273]:
#Return the first 5 rows of data
data.head(5)

,Unnamed: 0,datetime,load,apparent_temperature,temperature,humidity,dew_point,wind_speed,cloud_cover,date
0,0,2018-01-01 00:00:00,803.22270,10.45800,10.45800,0.955500,8.946000,0.0,0.0,2018-01-01
1,1,2018-01-01 00:15:00,774.89523,10.32675,10.32675,0.961625,8.911875,0.0,0.0,2018-01-01
2,2,2018-01-01 00:30:00,731.46927,10.19550,10.19550,0.967750,8.877750,0.0,0.0,2018-01-01
3,3,2018-01-01 00:45:00,713.93870,10.06425,10.06425,0.973875,8.843625,0.0,0.0,2018-01-01
4,4,2018-01-01 01:00:00,699.23007,9.93300,9.93300,0.980000,8.809500,0.0,0.0,2018-01-01


In [274]:
# Confirm the date column is in datetime format
data['datetime']=pd.to_datetime(data['datetime'],format= "%Y-%m-%d %H:%M:%S") 

In [275]:
#Refine the datetime attribute to increase the accurecy of the model
data['year']=data['datetime'].dt.year
data['month']=data['datetime'].dt.month
data['day']=data['datetime'].dt.day
data['hour']=data['datetime'].dt.hour
data['minutes']=data['datetime'].dt.minute
data['week']=data['datetime'].dt.week
data['weekofyear']=data['datetime'].dt.weekofyear
data['dayofweek']=data['datetime'].dt.dayofweek
data['weekday']=data['datetime'].dt.weekday
data['quarter']=data['datetime'].dt.quarter

<ipython-input-275-93e0aa8de5b6>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['week']=data['datetime'].dt.week
<ipython-input-275-93e0aa8de5b6>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['weekofyear']=data['datetime'].dt.weekofyear


In [276]:
#Task 2-> Nan values filtering
Nan_Val=data['load'].isna()
nan_pred=data.loc[Nan_Val]
X_Nan_Pred=nan_pred[['apparent_temperature','temperature','humidity','dew_point','wind_speed','cloud_cover','month','day','hour','minutes','week','weekofyear','dayofweek','weekday','quarter']]
y_Nan_Pred=nan_pred[['load']]
X_Nan_Pred=pd.DataFrame(X_Nan_Pred)

In [ ]:
data.dropna(inplace=True)
X=data[['apparent_temperature','temperature','humidity','dew_point','wind_speed','cloud_cover','month','day','hour','minutes','week','weekofyear','dayofweek','weekday','quarter']]
y=data[['load']]
gbr = GradientBoostingRegressor()
model = MultiOutputRegressor(estimator=gbr)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=500) # 70% training and 30% test
print(model)
model.fit(X_train, y_train)
train_score=(model.score(X_train, y_train))
print("Model training accurecy is: ",train_score*100,'%')
yhat = model.predict(X_test)
ynan = model.predict(X_Nan_Pred)
nan_pred=pd.DataFrame(nan_pred)
df=pd.DataFrame()
df['datetime']=nan_pred['datetime']
df['forecast']= np.array(ynan)
#Task 1 - Do a short term forecast for the day 14 December 2020 in the frequency of 15 minutes.
New_Test_Data=[[10.45800,10.45800,0.955500,8.946000,0.0,0.0,12,14,0.0,15,2,1,4,1,4]]
x={'datetime':"2020-12-14 00:15:00",'forecast':model.predict(New_Test_Data)[0][0]}
df=df.append(x, ignore_index = True)
df.to_csv('Output_Load_forecasting.csv',index=False)
test_score=(model.score(X_test,yhat))
print("Model test accurecy is: ",(model.score(X_test,yhat))*100,'%')


MultiOutputRegressor(estimator=GradientBoostingRegressor())


In [ ]:
sns.set_theme(style="whitegrid")

# Draw a scatter plot while assigning point colors and sizes to different
# variables in the dataset
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
clarity_ranking = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]
sns.scatterplot(x="datetime", y="load",
                hue="month", size="year",
                palette="ch:r=-.2,d=.3_r",
                hue_order=clarity_ranking,
                sizes=(1, 10), linewidth=0,
                data=data, ax=ax)
plt.savefig('d2.png')

In [ ]:
sns.set_theme(style="white")

# Plot miles per gallon against horsepower with other semantics
sns.relplot(x="datetime", y="load", hue="year", size="month",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=10, data=data)
plt.savefig('books_read.png')

In [269]:
from sklearn.model_selection import validation_curve
from sklearn.ensemble import RandomForestClassifier

param_range = np.arange(1, 250, 2)

train_scores, test_scores = validation_curve(MultiOutputRegressor(),X, y, param_name="estimator", param_range=param_range,cv=4, scoring="accuracy", n_jobs=-1)

train_score, test_score = validation_curve(KNeighborsClassifier(), X, y,
                                       param_name = "n_neighbors",
                                       param_range = parameter_range,
                                        cv = 5, scoring = "accuracy")

#train_score=np.argmax(train_score, axis=1)
#test_score=np.argmax(test_score, axis=1)
train_mean = np.mean(train_score, axis=0)
train_std = np.std(train_score, axis=0)
test_mean = np.mean(test_score, axis=1)
test_std = np.std(test_score, axis=1)

plt.subplots(1, figsize=(7,7))
plt.plot(param_range, train_mean, label="Training score", color="black")
plt.plot(param_range, test_mean, label="Cross-validation score", color="dimgrey")

plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, color="gray")
plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, color="gainsboro")
 
plt.title("Validation Curve With Random Forest")
plt.xlabel("Number Of Trees")
plt.ylabel("Accuracy Score")
plt.tight_layout()
plt.legend(loc="best")
plt.show()

[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan na